In [1]:
#instalação de pacotes
#!pip3 install pyramid-arima
!pip install pmdarima
!pip install mysql-connector-python
!pip install keras

# Seleciona Produtos

In [110]:
import numpy as np
import pandas as pd
import mysql.connector

#Conecta com banco MySQL
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='projeto',
                                         user='root',
                                         password='12345678')
    
    sql_select_Query = " select sk_produto \
                        from dim_produto as pd \
                        where pd.sk_produto > 0 "
    
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    
    #carrega dados
    records = cursor.fetchall()

    data_sql = pd.DataFrame(records)

except mysql.connector.Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if connection.is_connected():
        connection.close()
        cursor.close()
        print("MySQL connection is closed")
        
df_prod = data_sql.rename(columns={0: "sku"})
        
df_prod.info()

MySQL connection is closed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   sku     7 non-null      int64
dtypes: int64(1)
memory usage: 184.0 bytes


In [111]:
def movimento_real(produto):
    
    #Conecta com banco MySQL
    try:
        connection = mysql.connector.connect(host='localhost',
                                             database='projeto',
                                             user='root',
                                             password='12345678')

        #produto_proc = "12804"

        sql_select_Query = " select ds_data_sem_hora \
                            ,isbn_produto \
                            ,qtd_venda  \
                            from fato_venda as ft \
                            , dim_tempo as tp \
                            , dim_produto as pd \
                            where tp.sk_tempo = ft.sk_tempo \
                            and pd.sk_produto = ft.sk_protudo \
                            and pd.sk_produto = " + produto 

        cursor = connection.cursor()
        cursor.execute(sql_select_Query)

        #carrega dados
        records = cursor.fetchall()

        data_sql = pd.DataFrame(records)

    except mysql.connector.Error as e:
        print("Error reading data from MySQL table", e)
    finally:
        if connection.is_connected():
            connection.close()
            cursor.close() 
            print("MySQL connection is closed")

    if len(data_sql)>0:
        
        print("data_sql>0")
        
        #ajusta dados para a predição
        df_mov = data_sql.copy()

        #adiciona nome nas colunas
        df_mov = df_mov.rename(columns={0: "date",1:"isbn", 2: "value"})

        #transforma para datetime
        df_mov["date"] = pd.to_datetime(df_mov["date"])

        #ordena
        df_mov = df_mov.sort_values('date')

        #soma quantidade em datas iguais
        df_mov = df_mov.groupby('date')['value'].sum().reset_index()

        #transforma a data em indice
        df_mov = df_mov.set_index('date')

        #agrupa na soma do mês
        df_mov = df_mov['value'].resample('MS').sum()
    else: 
        
        print("data_sql=0")
        
        df_mov = []
    
    return df_mov


In [112]:
import pmdarima as pm

def predicao_arima(df_arima,n_periods):
    
    #Separa dados
    train = df_arima.iloc[:len(df_arima)-12]
    test = df_arima.iloc[len(df_arima)-12:]

    #Fit modelo auto-arima
    fitSArima = pm.auto_arima(df_arima, start_p=1, start_q=1,max_p=3, max_q=3, m=12,
                             start_P=0, seasonal=True, d=None, D=1, trace=False,
                             error_action='ignore',suppress_warnings=True, stepwise=True)

    fitSArima.fit(train)
    
    #Forecast auto-arima
    index_pred = pd.date_range(test.index[0], periods = n_periods, freq='MS')

    predicao, confint = fitSArima.predict(n_periods=n_periods, return_conf_int=True)

    #Cria dataseries
    predicao = pd.DataFrame(predicao,index = index_pred,columns=['SARIMA'])
    
    return predicao

In [113]:
from fbprophet import Prophet

def predicao_prophet(df_prophet,n_periods):
    
    #Quantida de períodos futuros
    fut_per = int(n_periods/2)
    
    #Ajust dados
    df_prophet = df_prophet.to_frame()

    #Reinicia indice
    df_prophet = df_prophet.reset_index()

    #adiciona nome nas colunas
    df_prophet = df_prophet.rename(columns={"date":"ds","value":"y"})
    
    #Fit modelo Prophet
    fitProphet = Prophet()

    fitProphet.fit(df_prophet)
    
    #Forecast Prophet
    future = fitProphet.make_future_dataframe(periods=fut_per, freq='MS')

    predProphetfull = fitProphet.predict(future)

    predicao = predProphetfull[['ds','yhat']]

    predicao = predicao.set_index('ds')
    predicao = predicao.rename(columns={"yhat": "PROPHET"})

    predicao = predicao.iloc[-24:]

    return predicao

In [ ]:
#Numero de períodos para predição
n_periods = 24

for ind in df_prod.index:
    
    print("produto " + str(int(df_prod['sku'][ind])) )
          
    #Carrega movimentos do produto
    df1 = movimento_real(str(int(df_prod['sku'][ind])).strip())
    
    if len(df1)>0:
        
        #Trata outliers superiores 
        #df = df.where(((df-df.mean()).abs() < 3*df.std()),3*df.std())
        df1 = df1.where(df1<df1.quantile(0.95),df1.quantile(0.95))

        #Calcula Predição - Modelo SArima
        predSArima = predicao_arima(df1,n_periods)

        predProphet = predicao_prophet(df1,n_periods)
        
        #Concatena predições
        test = df1.iloc[len(df1)-12:]
        forecast = pd.concat([test,predSArima,predProphet],axis=1)
        
        #forecast['Ac_venda'] = test.cumsum()
        #forecast['Ac_SArima'] = forecast['SARIMA'].cumsum()
        #forecast['Ac_PROPHET'] = forecast['PROPHET'].cumsum()
        #forecast['Ac_LSTM'] = forecast['LSTM'].cumsum()

        print(forecast)

produto 5795
MySQL connection is closed
data_sql>0


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


             value       SARIMA     PROPHET
2019-01-01   739.0   101.165368  302.875817
2019-02-01   387.0   359.074585  374.076548
2019-03-01   589.0   961.031302  672.736738
2019-04-01   341.0   178.055034  250.594077
2019-05-01   166.0   539.405606  249.952491
2019-06-01  1232.3   471.596938  698.139139
2019-07-01  1085.0   744.744969  731.889774
2019-08-01   224.0   448.416760  323.058982
2019-09-01   485.0   339.112300  684.475035
2019-10-01   304.0   313.803642  311.404569
2019-11-01   144.0   230.960052  227.349988
2019-12-01   320.0  1086.008939  875.030780
2020-01-01     NaN    28.030665  327.409577
2020-02-01     NaN   302.096981  327.818150
2020-03-01     NaN   652.884254  670.511293
2020-04-01     NaN    86.804796  222.656354
2020-05-01     NaN   186.214525  254.405397
2020-06-01     NaN   278.508018  577.767982
2020-07-01     NaN   811.148847  604.406774
2020-08-01     NaN   465.370123  656.265249
2020-09-01     NaN   172.951878  191.886737
2020-10-01     NaN   132.465339 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


            value      SARIMA     PROPHET
2019-01-01   40.0  195.859519  186.747988
2019-02-01   14.0  -26.330954  -12.650818
2019-03-01  380.0  379.249149  328.816525
2019-04-01   71.0  156.759695  179.859908
2019-05-01  263.0   50.120002  180.404164
2019-06-01  116.0  325.850117  174.249645
2019-07-01   77.0  160.461706  160.978032
2019-08-01   99.0  285.234103  245.921244
2019-09-01  387.0   86.741735  264.335062
2019-10-01  175.0  225.269617  143.864405
2019-11-01   59.0   12.999595   14.037451
2019-12-01  289.0   32.825990  103.956501
2020-01-01    NaN  143.976365  222.353218
2020-02-01    NaN  -32.677158  -54.481768
2020-03-01    NaN  194.885501  393.532431
2020-04-01    NaN  217.479838  142.737401
2020-05-01    NaN   21.019338   29.979920
2020-06-01    NaN  266.673129  270.668329
2020-07-01    NaN  134.049622  151.015528
2020-08-01    NaN  296.604884  248.195247
2020-09-01    NaN   35.891216   78.170585
2020-10-01    NaN  197.057919  246.851612
2020-11-01    NaN   -0.294819   76

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
